# Analysis of AFM test on turgid and plasmolyzed BY2 cells

## Last Updated: January 23, 2020

This document was prepared by Leah Ginsberg, a member of the [Ravichandran Research Group](https://www.ravi.caltech.edu/) at [Caltech](http://www.caltech.edu) in collaboration with [Professor Eleftheria Roumeli](https://sites.google.com/uw.edu/roumeli-research-group/) from [University of Washington](https://www.washington.edu/). 

In [1]:
import numpy as np               # general math operations
import pandas as pd              # pandas dataframes
import os                        # for opening .csv files in different folders
import altair as alt             # pretty plotting
import altair_catplot as altcat  # pretty box & whisker plots
from scipy import signal, stats, fftpack  # filtering and statistics
import matplotlib.pyplot as plt  # plotting histogram

alt.data_transformers.enable('data_server') # converts data to json for handling large data sets

DataTransformerRegistry.enable('data_server')

In [2]:
df=pd.read_csv('AFM_results.csv',usecols=[0,1,2,3,4,5])
df.head()

,Media,E (MPa),Indentation,Force,Spring Constant,"Linear Fit (b, ax)"
0,Di Water,1.41,80.09,33.72,15.21 nN/nm,"-2.5429e-012,0.30987"
1,Di Water,1.31,82.49,33.82,15.21 nN/nm,"-5.7893e-010,0.36601"
2,Di Water,1.80,47.30,61.34,15.21 nN/nm,"-5.6686e-010,0.67858"
3,Di Water,2.55,77.36,35.29,15.21 nN/nm,"-3.4613e-009,0.28384"
4,Di Water,3.34,35.04,49.50,15.21 nN/nm,"1.5904e-009,0.29944"


In [3]:
# report summary statistics
sumstats = df[['Media','E (MPa)']].groupby('Media').describe()
sumstats

E (MPa)                                                   \
                    count      mean       std      min       25%       50%   
Media                                                                        
Di Water             34.0  5.273965  3.842958  0.92482  1.805000  3.045000   
Growth Media          9.0  6.296667  3.203326  1.27000  5.830000  6.650000   
LatB/C2              38.0  2.188742  0.695630  0.71219  1.780000  2.230000   
LatB/sorbitol        21.0  0.131962  0.060154  0.02700  0.100420  0.118580   
oryzalin/C2          31.0  4.175058  3.414976  0.64663  1.545000  4.260000   
oryzalin/sorbitol    18.0  0.295916  0.121740  0.09736  0.181290  0.319750   
sorbitol             22.0  0.274655  0.116644  0.10247  0.173157  0.257855   

                                      
                       75%       max  
Media                                 
Di Water           8.50500  13.79000  
Growth Media       8.25000  10.84000  
LatB/C2            2.49250   3.50000  
LatB/sorbitol      0.17753   0.24151  
oryzalin/C2        5.92500  15.22000  
oryzalin/sorbitol  0.41575   0.44214  
sorbitol           0.36930   0.49153

In [4]:
min(df[df['Media']=='sorbitol']['E (MPa)'])

0.10247

In [5]:
min(df[df['Media']=='LatB/sorbitol']['E (MPa)'])

0.027000000000000003

In [6]:
min(df[df['Media']=='oryzalin/sorbitol']['E (MPa)'])

0.09736

In [7]:
SE_GM = 3.203326/np.sqrt(9)
SE_GMMT = 3.414976/np.sqrt(31)
SE_GMAF = 0.695630/np.sqrt(38)
SE_PS = 0.274655/np.sqrt(22)
SE_PSMT = 0.121740/np.sqrt(18)
SE_PSAF = 0.131962/np.sqrt(21)
SE_GM, SE_GMMT, SE_GMAF, SE_PS, SE_PSMT, SE_PSAF

(1.0677753333333333,
 0.6133477958941876,
 0.11284608718119235,
 0.05855664275065018,
 0.0286943931805501,
 0.028796469231322026)

## YM analysis

In [8]:
ECDFs = altcat.catplot(data=df[df['Media']=='Growth Media'],
               height=350,
               width=450, 
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('red')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/C2'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('yellow')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='LatB/C2'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('orange')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('green')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='LatB/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('blue')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('E (MPa):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('purple')},
               transform='ecdf')

ECDFs.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [9]:
boxjitter = altcat.catplot(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('E (MPa):Q', scale=alt.Scale(domain=(0,20))),
                             y=alt.Y('Media:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('Media:N', scale=alt.Scale(domain=['Growth Media', 'LatB/C2', 'LatB/sorbitol','oryzalin/C2','oryzalin/sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [10]:
boxjitter = altcat.catplot(df[(df['Media']!='Di Water')&(df['Media']!='Growth Media')&(df['Media']!='LatB/C2')&(df['Media']!='oryzalin/C2')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('E (MPa):Q', scale=alt.Scale(domain=(0,0.6))),
                             y=alt.Y('Media:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('Media:N', scale=alt.Scale(domain=['Growth Media', 'LatB/C2', 'LatB/sorbitol','oryzalin/C2','oryzalin/sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

alt.LayerChart(...)

In [8]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(), 
                df[df['Media']=='oryzalin/C2']['E (MPa)'].to_numpy())
pKS

0.0490767077454386

In [9]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/C2']['E (MPa)'].to_numpy())
pKS

5.8334898819345504e-05

In [10]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/C2']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/C2']['E (MPa)'].to_numpy())
pKS

0.00010517581586122482

In [11]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='oryzalin/sorbitol']['E (MPa)'].to_numpy())
pKS

0.7556643975221443

In [12]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['E (MPa)'].to_numpy())
pKS

0.0014764941542995125

In [13]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['E (MPa)'].to_numpy())
pKS

0.00011437690679549917

In [17]:
[T, pT] = stats.ttest_ind(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(), 
                df[df['Media']=='oryzalin/C2']['E (MPa)'].to_numpy())
pT

0.1047568844634888

In [18]:
[T, pT] = stats.ttest_ind(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/C2']['E (MPa)'].to_numpy())
pT

2.327992631205129e-09

## k analysis

In [24]:
ax_vec = [float(df['Linear Fit (b, ax)'][i].split(',')[1]) for i in range(len(df))]
df['kCW (N/m)'] = ax_vec
df

,Media,E (MPa),Indentation,Force,Spring Constant,"Linear Fit (b, ax)",kCW (N/m)
0,Di Water,1.41,80.09,33.72,15.21 nN/nm,"-2.5429e-012,0.30987",0.30987
1,Di Water,1.31,82.49,33.82,15.21 nN/nm,"-5.7893e-010,0.36601",0.36601
2,Di Water,1.80,47.30,61.34,15.21 nN/nm,"-5.6686e-010,0.67858",0.67858
3,Di Water,2.55,77.36,35.29,15.21 nN/nm,"-3.4613e-009,0.28384",0.28384
4,Di Water,3.34,35.04,49.50,15.21 nN/nm,"1.5904e-009,0.29944",0.29944
...,...,...,...,...,...,...,...
168,LatB/C2,3.02,75.34,65.97,10.14 nN/nm,"1.815e-011,0.48653",0.48653
169,LatB/C2,2.36,90.22,68.04,10.14 nN/nm,"1.2752e-009,0.37883",0.37883
170,LatB/C2,2.11,91.43,58.55,10.14 nN/nm,"-1.7288e-009,0.44677",0.44677
171,LatB/C2,1.98,90.01,55.82,10.14 nN/nm,"-8.443e-010,0.43179",0.43179


In [25]:
sumstats = df[['Media','kCW (N/m)']].groupby('Media').describe()
sumstats

kCW (N/m)                                                    \
                      count      mean       std       min       25%       50%   
Media                                                                           
Di Water               34.0  0.589060  0.344647  0.185930  0.289592  0.448635   
Growth Media            9.0  0.694841  0.333870  0.142790  0.469070  0.754890   
LatB/C2                38.0  0.429510  0.153535  0.174570  0.321240  0.389140   
LatB/sorbitol          21.0  0.053442  0.037724  0.017532  0.033365  0.047440   
oryzalin/C2            31.0  0.523036  0.299170  0.129430  0.261375  0.454070   
oryzalin/sorbitol      18.0  0.127047  0.056134  0.037264  0.084946  0.124925   
sorbitol               22.0  0.094532  0.055572  0.035201  0.055534  0.080982   

                                      
                        75%      max  
Media                                 
Di Water           0.925245  1.41040  
Growth Media       0.906470  1.15800  
LatB/C2            0.482745  0.93879  
LatB/sorbitol      0.057845  0.20158  
oryzalin/C2        0.745470  1.19200  
oryzalin/sorbitol  0.153612  0.24082  
sorbitol           0.116187  0.24233

In [36]:
meankCW_GMall = np.mean(df[(df['Media']=='Growth Media') | (df['Media']=='LatB/C2') | (df['Media']=='oryzalin/C2')]['kCW (N/m)'])

In [37]:
meankCW_PSall = np.mean(df[(df['Media']=='sorbitol') | (df['Media']=='LatB/sorbitol') | (df['Media']=='oryzalin/sorbitol')]['kCW (N/m)'])

In [38]:
meankCW_GMall/meankCW_PSall

5.526694420601892

In [15]:
ECDFs = altcat.catplot(data=df[df['Media']=='Growth Media'],
               height=350,
               width=450, 
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('red')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/C2'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('yellow')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='LatB/C2'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('orange')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('green')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='LatB/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('blue')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW stiffness (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('purple')},
               transform='ecdf')

ECDFs.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

KeyError: 'CW stiffness (N/m)'

In [16]:
boxjitter = altcat.catplot(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('CW stiffness (N/m):Q', scale=alt.Scale(domain=(0,2))),
                             y=alt.Y('Media:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('Media:N', scale=alt.Scale(domain=['Growth Media', 'LatB/C2', 'LatB/sorbitol','oryzalin/C2','oryzalin/sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

KeyError: 'CW stiffness (N/m)'

In [ ]:
boxjitter = altcat.catplot(df[(df['Media']!='Di Water')&(df['Media']!='Growth Media')&(df['Media']!='LatB/C2')&(df['Media']!='oryzalin/C2')],
               height=175,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(x=alt.X('CW stiffness (N/m):Q', scale=alt.Scale(domain=(0,0.3))),
                             y=alt.Y('Media:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('Media:N', scale=alt.Scale(domain=['Growth Media', 'LatB/C2', 'LatB/sorbitol','oryzalin/C2','oryzalin/sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='oryzalin/C2']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='LatB/C2']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/C2']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='LatB/C2']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='oryzalin/sorbitol']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/sorbitol']['CW stiffness (N/m)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['CW stiffness (N/m)'].to_numpy())
pKS

In [ ]:
for i,item in enumerate(df['Linear Fit (b, ax)']):
    df.loc[i,'Linear Fit (b, ax)']=float(item.split(',')[1])
for i,item in enumerate(df['Spring Constant']):
    df.loc[i,'Spring Constant']=float(item.split(' ')[0])

In [ ]:
for i,k1 in enumerate(df['Spring Constant']):
    keq = df.iloc[i]['Linear Fit (b, ax)']
    k2 = 1/(1/keq-1/k1)
    df.loc[i,'CW spring (N/m)'] = k2

In [ ]:
ECDFs = altcat.catplot(data=df[df['Media']=='Growth Media'],
               height=350,
               width=450, 
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW spring (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('red')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/C2'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW spring (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('yellow')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW spring (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('green')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='LatB/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW spring (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('blue')},
               transform='ecdf')
ECDFs += altcat.catplot(data=df[df['Media']=='oryzalin/sorbitol'],
               mark=dict(opacity=0.8, type='point'),
               encoding={'x': alt.X('CW spring (N/m):Q', scale=alt.Scale(zero=False)),
                         'fill': alt.value('purple')},
               transform='ecdf')

ECDFs.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

In [ ]:
boxjitter = altcat.catplot(df[(df['Media']!='Di Water')&(df['Media']!='Growth Media')&(df['Media']!='oryzalin/C2')],
               height=350,
               width=450, 
               mark=dict(opacity=1.0, type='point'),
               box_mark=dict(strokeWidth=5, opacity=0.75, color='#abb0ac'),
               whisker_mark=dict(strokeWidth=2, opacity=0.75, color='#bababa'),
               encoding=dict(y=alt.Y('CW spring (N/m):Q', scale=alt.Scale(domain=(0,0.5))),
                             x=alt.X('Media:N'),
#                              # For black&white version
#                              fill=alt.Color('solution:N', scale=alt.Scale(domain=['T87', 'sorbitol', 'water'], range=['#808080', '#000000', '#FFFFFF'])),
#                              # For colored version
                             fill=alt.Color('Media:N', scale=alt.Scale(domain=['Growth Media', 'LatB/C2', 'LatB/sorbitol','oryzalin/C2','oryzalin/sorbitol','sorbitol'], range=['red', 'orange', 'blue','yellow','purple','green']))),
               transform='jitterbox'
)
boxjitter.configure_text(fontWeight='bold', fontSize=20
).configure_legend(labelFontSize=20, titleFontSize=15
).configure_axis(labelFontSize=20, titleFontSize=20
).configure_mark(color='black'
).configure_point(size=100, strokeWidth=1)

In [62]:
# stat sign difference between all GM and all PS cells
[KS, pKS] = stats.ks_2samp(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')]['E (MPa)'].to_numpy(), 
                df[(df['Media']=='sorbitol')|(df['Media']=='LatB/sorbitol')|(df['Media']=='oryzalin/sorbitol')]['E (MPa)'].to_numpy())
pKS

4.440892098500626e-16

In [63]:
# GM vs GM-MT
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(),
                df[df['Media']=='oryzalin/C2']['E (MPa)'].to_numpy())
pKS

0.0490767077454386

In [64]:
# GM vs GM-AF
[KS, pKS] = stats.ks_2samp(df[df['Media']=='Growth Media']['E (MPa)'].to_numpy(),
                df[df['Media']=='LatB/C2']['E (MPa)'].to_numpy())
pKS

5.8334898819345504e-05

In [65]:
# GM-MT vs GM-AF
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/C2']['E (MPa)'].to_numpy(),
                df[df['Media']=='LatB/C2']['E (MPa)'].to_numpy())
pKS

0.00010517581586122482

In [66]:
# PS vs PS-MT
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='oryzalin/sorbitol']['E (MPa)'].to_numpy())
pKS

0.7556643975221443

In [67]:
# PS vs PS-AF
[KS, pKS] = stats.ks_2samp(df[df['Media']=='sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['E (MPa)'].to_numpy())
pKS

0.0014764941542995125

In [68]:
# PS-AF vs PS-MT
[KS, pKS] = stats.ks_2samp(df[df['Media']=='oryzalin/sorbitol']['E (MPa)'].to_numpy(), 
                df[df['Media']=='LatB/sorbitol']['E (MPa)'].to_numpy())
pKS

0.00011437690679549917

In [52]:
np.std(df['Indentation'])/np.sqrt(len(df))

7.861052090176416

In [53]:
np.mean(df['Indentation'])

126.73294797687868

In [55]:
np.mean(df[(df['Media']=='sorbitol')|(df['Media']=='LatB/sorbitol')|(df['Media']=='oryzalin/sorbitol')]['E (MPa)'])

0.23180508196721314

In [56]:
np.std(df[(df['Media']=='sorbitol')|(df['Media']=='LatB/sorbitol')|(df['Media']=='oryzalin/sorbitol')]['E (MPa)'])/np.sqrt(len(df[(df['Media']=='sorbitol')|(df['Media']=='LatB/sorbitol')|(df['Media']=='oryzalin/sorbitol')]))

0.015841223271044525

In [57]:
np.mean(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')]['E (MPa)'])

3.4521664102564102

In [58]:
np.std(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')]['E (MPa)'])/np.sqrt(len(df[(df['Media']=='Growth Media')|(df['Media']=='LatB/C2')|(df['Media']=='oryzalin/C2')]))

0.31391519161166476